In [9]:
!pip freeze | grep scikit-learn
# !pip install scikit-learn==1.5.0

scikit-learn==1.5.0


In [10]:
!python -V

Python 3.11.9


In [11]:
import pickle
import pandas as pd

In [12]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [13]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [14]:
year = 2023
month = 3
input_path = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet'
output_path = f'output/yellow_tripdata_{year:04d}-{month:02d}.parquet'

In [15]:
df = read_data(input_path)

In [16]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

<h3>Q1. Notebook. The standard deviation of the predicted duration for this dataset is</h3>

In [17]:
y_pred.std()

np.float64(6.247488852238703)

<h3>Q2. Preparing the output</h3>

In [18]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_output = pd.DataFrame()
df_output['ride_id'] = df['ride_id']
df_output['predicted_duration'] = y_pred

df_output.to_parquet(
    output_path,
    engine='pyarrow',
    compression=None,
    index=False
)

!ls -lh output

total 458776
-rw-r--r--  1 pavel_ozhych  staff    65M Oct  7 19:13 yellow_tripdata_2023-03.parquet
-rw-r--r--  1 pavel_ozhych  staff    63M Oct  7 17:37 yellow_tripdata_2023-04.parquet
-rw-r--r--  1 pavel_ozhych  staff    67M Oct  7 18:21 yellow_tripdata_2023-05.parquet


<h3>Q3. Creating the scoring script</h3>

In [4]:
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1727 bytes to starter.py
